In [ ]:
!git clone https://github.com/google-a2a/a2a-samples.git

fatal: destination path 'a2a-samples' already exists and is not an empty directory.


In [ ]:
%%shell
cd /content/
cd a2a-samples/samples/python

# uv 설치
curl -LsSf https://astral.sh/uv/install.sh | sh
uv sync

downloading uv 0.7.13 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
error: Failed to parse `uv.lock`
  Caused by: Dependency `grpcio-reflection` has missing `source` field but has more than one matching package


CalledProcessError: Command 'cd /content/
cd a2a-samples/samples/python

# uv 설치
curl -LsSf https://astral.sh/uv/install.sh | sh
uv sync
' returned non-zero exit status 2.

In [1]:
!pip install openai

In [62]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://110.44.51.165:8501/generate",
    api_key="token-abc123",
)

In [63]:
def generate_response(message_list):
    completion = client.chat.completions.create(
        model = "meta-llama/Llama-3.2-3B-Instruct",
        messages = message_list,
        max_tokens=1024,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )

    final_answer = []
    assistant_response = ""
    print("========= Llama-3.2-3B-Instruct ===========")

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [64]:
def generate_peft_response(message_list):
    completion = client.chat.completions.create(
        model = "llama_peft",
        messages = message_list,
        max_tokens=1024,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )

    final_answer = []
    assistant_response = ""
    print("========= llama_peft ===========")

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [53]:
def generate_completion_response(message_list):
    # Convert message list to a single string prompt
    prompt = "\n".join([f"{msg['role']}: {msg['content']}" for msg in message_list])

    completion = client.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",  # Note: Model names for completion might differ
        prompt=prompt,
        max_tokens=1024,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )

    final_answer = []
    assistant_response = ""
    print("========= Llama-3.2-3B-Instruct (Completion API) ===========")

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].text

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [65]:
message_list = [
    {"role": "system", "content": "당신은 유저의 질문에 최대한 정확하고 풍부한 정보를 전달하는 assistant 이다. 답변은 항상 한국어로 공손하게 답변해줘."},
    {"role": "user", "content": "안심번호와 함께 주문한 음식이 취소된 경우 어떻게 되나요? [배민 데이터 참조]"}
]

generate_response(message_list)
generate_completion_response(message_list)

========= Llama-3.2-3B-Instruct ===========
안녕하세요! 배달의민족 데이터를 바탕으로 안심번호와 함께 주문하신 음식이 취소되었을 경우의 상황을 자세히 안내해 드리겠습니다.

**안심번호와 음식 취소 시 환불 및 처리 과정**

배달의민족 안심번호는 고객님의 결제를 보호하는 시스템입니다. 안심번호를 통해 결제하신 후 주문이 취소된 경우, 일반적으로 다음과 같은 과정을 거쳐 환불이 진행됩니다.

1.  **취소 발생 시 알림:** 먼저, 음식 취소가 발생하면 배달의민족 앱을 통해 푸시 알림 또는 문자 메시지로 알려드립니다. 취소 사유와 함께 안내를 받을 수 있습니다.

2.  **환불 자동 처리:** 안심번호를 통해 결제하신 경우, 대부분의 경우 환불이 자동적으로 진행됩니다. 
    *   **결제 수단별 환불:**
        *   **신용/체크카드:** 카드사에서 환불을 진행하며, 보통 2~3일(영업일 기준) 정도 소요될 수 있습니다.
        *   **계좌이체:** 등록된 계좌로 환불이 진행되며, 보통 1~2일(영업일 기준) 정도 소요될 수 있습니다.
        *   **휴대폰 결제:** 통신사에서 환불을 진행하며, 보통 3~5일(영업일 기준) 정도 소요될 수 있습니다.
        *   **배민페이/배민카드:** 배민페이 잔액으로 환불이 되거나, 연결된 계좌로 환불이 진행됩니다.

3.  **환불 확인:** 배달의민족 앱 내의 '주문 내역' 또는 '안심번호' 메뉴에서 환불 진행 상황을 확인할 수 있습니다. 환불이 완료되면 '환불 완료' 상태로 표시됩니다.

4.  **환불 지연 시 대처:**
    *   **배달의민족 고객센터 문의:** 환불이 예정된 기간 내에 이루어지지 않을 경우, 배달의민족 고객센터(1588-9500)에 문의하여 환불 진행 상황을 확인하고 도움을 받을 수 있습니다.
    *   **결제 수단사에 문의:** 카드사, 은행 등 결제 수단사에 직접 문의하여 환불 진행 상황을 확인하는

"안녕하세요! 안심번호와 함께 주문하신 음식이 취소된 경우, 배달의민족(배민) 데이터에 따르면 다음과 같이 처리됩니다.\n\n**1. 결제 금액 환불:**\n\n*   **안심번호 결제:** 안심번호로 결제하신 경우, 음식 취소 시 결제는 자동으로 취소됩니다. 환불은 일반적으로 다음과 같은 과정을 거칩니다.\n    *   **즉시 환불:** 대부분의 경우, 음식점의 취소 요청이 접수되는 즉시 카드 또는 계좌로 환불이 진행됩니다.\n    *   **환불 처리 시간:** 환불 처리 시간은 카드사 또는 은행 정책에 따라 다를 수 있지만, 일반적으로 영업일 기준 1~3일 이내에 확인 가능합니다.\n    *   **환불 확인:** 배민 앱의 주문 내역에서 환불 상태를 확인할 수 있습니다.\n\n**2. 안심번호 해제:**\n\n*   음식 취소와 함께 안심번호도 자동으로 해제됩니다. 별도로 해제할 필요는 없습니다.\n*   만약 안심번호가 해제되지 않았다면, 배민 고객센터에 문의하여 해제를 요청할 수 있습니다.\n\n**3. 고객센터 문의:**\n\n*   환불이 지연되거나 문제가 발생한 경우, 배민 고객센터에 문의하면 자세한 안내를 받을 수 있습니다.\n    *   **배민 고객센터 연락처:** 앱 내 '고객문의' 또는 웹사이트([https://help.baemin.com/](https://help.baemin.com/))를 통해 문의 가능합니다.\n\n**4. 추가 정보:**\n\n*   음식점의 사정에 따라 취소 시기가 다를 수 있으며, 이 경우 환불 시점도 달라질 수 있습니다.\n*   취소 사유에 따라 환불 정책이 달라질 수 있습니다. 예를 들어, 음식점의 귀책사유로 취소된 경우와 고객의 요청으로 취소된 경우에 따라 환불 조건이 달라질 수 있습니다.\n\n**참고:**\n\n*   배민은 안심번호를 통해 안전한 결제 환경을 제공하며, 음식 취소 시에도 고객의 권익을 보호하기 위해 노력하고 있습니다.\n*   궁금한 점이 있다면 언제든지 배민 고객센터에 

In [ ]:
1import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_KEY')